In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df= pd.read_csv('../input/ecommerce-data/data.csv')

#copy data into new df for analysis
data=df.copy()

data.head()

In [ ]:
data.info()

### 1. Cohort Analysis:

We first divide the entire data into different cohorts to understand high-level trends. We use time cohort for our analysis here:

In [ ]:
#generate invoice month for each line purchase equal to the first day of the month when the purchase was made
data['InvoiceMonth'] = pd.to_datetime(data['InvoiceDate']).to_numpy().astype('datetime64[M]')

#first invoice month for every customer
data['CohortMonth'] = data.groupby('CustomerID')['InvoiceMonth'].transform('min')

#drop null values
data.dropna(inplace=True)

data.head()

In [ ]:
#drop NaN values
data.dropna()

#compute year and month from Invoice Date
invoice_year= data['InvoiceMonth'].dt.year.astype('int')
invoice_mon= data['InvoiceMonth'].dt.month.astype('int')

#compute year and month from Cohort Date
cohort_year= data['CohortMonth'].dt.year.astype('int')
cohort_mon= data['CohortMonth'].dt.month.astype('int')

#find the differences
diff_year = invoice_year - cohort_year
diff_mon = invoice_mon - cohort_mon

#calculate the cohort index for each invoice
data['CohortIndex'] = diff_year * 12 + diff_mon + 1
data.head()

In [ ]:
#group by cohort month and index and find number of unique customers for each grouping
grouped = data.groupby(['CohortMonth', 'CohortIndex'])['CustomerID'].apply(pd.Series.nunique)\
                                                                    .reset_index()
#pivot the data with cohort month as rows and Cohort Index as columns
grouped = grouped.pivot(index='CohortMonth', columns='CohortIndex',  values='CustomerID')
grouped

In the table above, the first column values represents the size of every individual cohort, and the subsequent columns represent the number of active customers for that cohort in the subsequent months.

In [ ]:
#divide each column by value of the first(cohort size) to find retention rate
size = grouped.iloc[:,0]
retention_table = grouped.divide(size, axis=0)

#compute the percentage
retention_table.round(3) * 100

Let's now visualize the retention rates on a heatmap:

In [ ]:
plt.figure(figsize=(10, 8))

sns.heatmap(data = retention_table,
            annot = True,        
            fmt = '.0%',         
            vmin = 0.0,          
            vmax = 0.5,           
            cmap = 'BuPu')
plt.show()

 ### 2. RFM Segmentation:
 RFM segmentation technique takes into account the past purchase behaviour and patterns of the customers to divide them into segments. Here 'R', 'F', 'M' denote Recency, Frequency and Monetary analysis respectively.
 
* Recency: Number of days since a customer's last purchase
* Frequency: Number of purchases by the customer
* Monetary: Total amount of money spent by the customer on his purchases
  
We will be sorting the customers into quartiles based on these three metrics, and calculating their RFM score. With the help of this score, we can effectively sort the customers into segments which can be used for targeting particular segments for campaigns, promotions or other personalized experiences.
 
For our ease of analysis, we'll be moving forward with the assumption that the date of our RFM analysis is  just the day after the most recent invoice date in the dataset.

In [ ]:
#copy data into a new dataframe for analysis
data_all= df.copy()

#calculating total the total amount for each line item (unit price * Quantity)
data_all['amount']= data_all['Quantity']* data_all['UnitPrice']
data_all.head()

#convert column to datetime
data_all['InvoiceDate']=pd.to_datetime(data_all['InvoiceDate'])

#setting date of analysis= 1 day after the most recent invoice
analysis_date = pd.to_datetime(data_all['InvoiceDate'].max())+ dt.timedelta(days=1)
data_all.head()

In [ ]:
#calculate the recency, frequency and Monetary values for each customer
grouped = data_all.groupby(['CustomerID'])\
                .agg({'InvoiceDate': lambda x: (analysis_date - x.max()).days,
                      'InvoiceNo': 'count',
                      'amount': 'sum'})\

#rename each column to denote the R,F,M Values
grouped.rename(columns = {'InvoiceDate': 'R_val',
                                   'InvoiceNo': 'F_val',
                                   'amount': 'M_val'}, inplace=True)
grouped.reset_index().head()

In [ ]:
#divide recency metric into 4 quartiles
r_quartiles = pd.qcut(grouped['R_val'], 4, labels = range(4, 0, -1))
grouped = grouped.assign(R_quartile = r_quartiles.values)

#divide frequency metric into 4 quartiles
f_quartiles = pd.qcut(grouped['F_val'], 4, labels = range(1, 5, 1))
grouped = grouped.assign(F_quartile = f_quartiles.values)

#divide monetary metric into 4 quartiles
m_quartiles = pd.qcut(grouped['M_val'], 4, labels = range(1, 5, 1))
grouped = grouped.assign(M_quartile = m_quartiles.values)

grouped.head()

Now that we have the R,F,M quartiles, we'll move on to calculating the RFM scores as a sum total of all the three quartiles and assigning RFM segments.

In [ ]:
def get_rfm_score(x):
    return str(x['R_quartile']) + str(x['F_quartile']) + str(x['M_quartile'])

#get RFM segment by concatenation of R,F,M values
grouped['RFM_Seg']= grouped.apply(get_rfm_score, axis=1)

#calculate RFM by summing R,F,M values
grouped['RFM_Score']= grouped[['R_quartile','F_quartile','M_quartile']].sum(axis=1)
grouped.head()

Visualizing the total count of each RFM Segment:

In [ ]:
#set sns theme
sns.set_theme(style="whitegrid")

#set plot size
fig, ax = plt.subplots(figsize=(20, 5))

#plot count of each RFM segment
sns.countplot(x="RFM_Seg", data=grouped)
plt.xticks(rotation=45)

plt.show()

We can clearly see that segment '111'(Worst customers) and segment '444'(best customers) have the highest count.

Now as per the business requirements, we can analyse each segment. 

First, we group the customers into different tiers based on their total RFM Score. In decreasing order of their RFM scores, they are as follows:
1. Platinum
2. Gold
3. Advanced
4. Basic

In [ ]:
def get_tier(a):
    if a >9:
        return 'Platinum'
    elif (a>6) & (a<=9):
        return 'Gold'
    elif (a>3) & (a<=6):
        return 'Advanced'
    elif (a>0) & (a<=3):
        return 'Basic' 

#assign a tier to each customer based on the get_tier function
grouped['Tier']=grouped['RFM_Score'].apply(get_tier)
grouped.head()

Next, let us compute the mean of the R-F-M values of each tier and also their counts.

In [ ]:
df_reset= grouped.reset_index()
tier_analysis=df_reset.groupby(['Tier'])\
        .agg({'R_val': 'mean',
                      'F_val': 'mean',
                      'M_val': 'mean'}).round(2)
print(tier_analysis,'\n')
print(grouped['Tier'].value_counts())
#['Basic','Advanced','Gold', 'Platinum']

We can see that the tiers have very distinctive properties, and they can be further analysed as per business needs.

A few takeaways:

1. Tiers with higher Recency values: These are the inactive customers. Surveys should be undertaken to understand their experience and pain-points with the store/app and appropriate measures should be taken in an effort to re-engage them

2. Tiers with lower Frequeny Values: These customers should be targeted with additional offers and campaigns from time to time to increase their frequency of purchase

3. Tiers with lower Monetary Values: For these customers, marketing and pricing strategies need to be formulated to increase their basket value. This can be achieved by offering discounts on a minimum cart price or bulk purchase discounts

Now, let's do an analysis of the tiers country-wise. This will help us understand which are the best and worst performing regions.

We will find out the top 5 countries with the largest percentage of the customers in the Basic Tier and the top 5 countries with the largest percentage of customers in the Platinum Tier.

In [ ]:
#find total customers in each country
country_count=data_all.groupby(['Country']).size().to_frame('Total_Customers_in_country').reset_index()
country_count.head()

#merge with original df to add the 'Country' column
df_with_country = df_reset.merge(data_all[['CustomerID','Country']], how='inner', on='CustomerID')
result= df_with_country.groupby(['Tier','Country']).size().to_frame('Customer_count').reset_index()
country_data= result.merge(country_count, how='inner', on='Country')

#get filtered dataset for Basic Tier
basic_tier= country_data[country_data['Tier'].isin(['Basic'])]

#calculate percentage of Basic Tier customers in each country
basic_tier['Basic_Tier_Percentage']= ((basic_tier['Customer_count']/basic_tier['Total_Customers_in_country'])*100)\
                                           .round(3)
#display top 5 countries with the largest percentage of Basic Tier Customers
basic_tier[['Country', 'Basic_Tier_Percentage']].sort_values('Basic_Tier_Percentage', ascending=False).head()

In [ ]:
#get filtered dataset for Platinum Tier
platinum_tier= country_data[country_data['Tier'].isin(['Platinum'])]

#calculate percentage of Platinum Tier customers in each country
platinum_tier['Platinum_Tier_Percentage']= ((platinum_tier['Customer_count']/platinum_tier['Total_Customers_in_country'])*100)\
                                            .round(3)
#display top 5 countries with the largest percentage of Platinum Tier Customers
platinum_tier[['Country','Platinum_Tier_Percentage']].sort_values('Platinum_Tier_Percentage', ascending=False).head()

From the above analysis, we can Singapore and Iceland have the most active and profitable customer base with 100% of them belonging to the Platinum Tier.

While, Saudi Arabia does not have any customer outside the Basic Tier.

### 3. K-Means Clustering:

Now, we will be approaching the segmentation using K-Means Clustering, a popular unsupervised learning algorithm. But before we start, we need to process the data to adhere to the following assumptions of K-Means Clustering with the techiques mentioned below:

1. K-Means assumes that the variables are not skewed. We will test our R,F,M values. If they are skewed, we will use logarithmic transformation to eliminate the skewness

2. K-Means assumes that all the variables have a similar mean and variance. Therefore, we will check the range and mean of each of the variables and if they are dissimilar, we will be using the Standard Scalar to normalize them

In [ ]:
#plot the skewness of recency metric
sns.distplot(grouped['R_val'])

In [ ]:
#plot the skewness of frequency metric
sns.distplot(grouped['F_val'])

In [ ]:
#plot the skewness of monetary metric
fig, ax = plt.subplots(figsize=(13, 7))
sns.distplot(grouped['M_val'])

In [ ]:
#log transformation of recency metric
recency_log= np.log(grouped['R_val'])

#plot the transformed variable
sns.distplot(recency_log)
plt.show()

In [ ]:
#check for variance and mean of the variables
grouped.describe()

In [ ]:
rfm = grouped[['R_val','F_val','M_val']]

#making all values in M_val positive
rfm['M_val']=rfm['M_val']+1

#applying logarithmic transformation
for c in ['R_val', 'F_val']:
    rfm[c]= np.log(rfm[c])
    

#Normalization of variables
from sklearn.preprocessing import StandardScaler

ecomm_standardized= StandardScaler().fit_transform(rfm)
rfm[['R_val','F_val','M_val']]=ecomm_standardized.round(2)

rfm.head()

The above step concludes the pre-processig of the data. Now, we use the elbow criterion method to find the optinum number of clusters.

In [ ]:
#start k-means clusterig
from sklearn.cluster import KMeans

sse = {}

#find the optimum number of clusters from 1 to 10
for k in range(1, 11):    
    kmeans = KMeans(n_clusters=k, random_state=1)    
    kmeans.fit(rfm)    
    sse[k] = kmeans.inertia_ 
    
# Plot SSE for each value of k    
plt.title('The Elbow Method')
plt.xlabel('k'); 
plt.ylabel('SSE')
sns.pointplot(x=list(sse.keys()), y=list(sse.values()))
plt.show()

In [ ]:
#fit k-means with 3 clusters
kmeans = KMeans(n_clusters=3, random_state=1)    
kmeans.fit(rfm)

#adding column with cluster labels to a new df
cluster_table = grouped.assign(Cluster=kmeans.labels_)

#group by cluster
clustered_data = cluster_table.groupby(['Cluster'])

#average RFM values for each cluster
clustered_data.agg({
    'R_val': 'mean',
    'F_val': 'mean',
    'M_val': 'mean'
  }).round(2)